In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
season_rpl = int(input('Сезон рпл: '))
tur_start = int(input('C какого тура: '))
tur_finish = int(input('По какой: '))

Сезон рпл: 2023
C какого тура: 1
По какой: 2


In [3]:
list_trans = []

for i in range(tur_start,tur_finish+1):
    
    # Получаю суп страницы 

    URL = f'https://football.kulichki.net/ruschamp/{season_rpl}/{i}/' # чемпионат, год, тур
    HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    data = requests.get(URL, headers=HEADERS)
#    print(data.status_code)
    soup = BeautifulSoup(data.text, 'html.parser')

    # Выделяю из супа ссылки тестовых трансляций кажого матча

    
    trans = soup.findAll('b')
    for i in trans:
        list_atr_a = i.find_all('a', href=True)
        for i in list_atr_a:
            list_contain_href = i['href']
            if 'trans' in list_contain_href:
                list_trans.append(list_contain_href)

In [4]:
print('Сколько матчей сыграно: ',len(list_trans))

Сколько матчей сыграно:  16


In [5]:
# Открываю ссылку, получаю суп страницы с текстом матча

site_found = 'https://football.kulichki.net/'
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

players = []
events = []

for match in list_trans:    
    data = requests.get(site_found+match, headers=HEADERS)
    soup = BeautifulSoup(data.text, 'html.parser')
#    print(data.status_code)
    
    # получаю кто играл и текст матча

    text = soup.findAll('tr')

    for i in text:    
        a = i.find_all('a', href=True)

        for i in a:
            if 'players' in i['href']:
                if '' != i.text:
                    players.append(i.text.strip())
    players=list(set(players))
    
    text = soup.findAll('tr')

    for i in text:    
        list_span = i.find_all('span')
        for i in list_span:
            if '' != i.text:
                events.append(i.text.strip())

In [6]:
# Делим события по запятым

events_divide=[]
for i in events:
    if ', ' in i:
        a=i.split(', ')
        
    elif '. ' in i:    
        a=i.split('. ')
     
    elif '! ' in i:    
        a=i.split('! ')
        
    else:
        a=i.split('... ')
    events_divide.append(a)

events_divide = list(set(sum(events_divide, [])))

In [7]:
# Анализирую действия игроков по тексту матча
s1=[]
s2=[]

for i in players:
    for j in events_divide:
        if i in j:
            s1.append(i)
            s2.append(j)


df=pd.DataFrame()
df['players'] = s1
df['events'] = s2

In [14]:
df.sample(5)

,players,events,pass,goal,shoot
262,Комличенко,получает по ногам Комличенко,0,0,0
838,Агаларов,который прострелил на Агаларова,0,0,0
556,Жоау Милсон,Жоау Милсон уже успел отметиться в атаке,0,0,0
96,Клаудиньо,после чего разыграли оного в касание слева от ...,0,0,0
783,Вячеслав Грулев,Вячеслав Грулев отправил мяч в сетку,0,0,0


In [9]:
df['players'].value_counts().head(10)

Изидор        32
Комличенко    32
Сперцян       24
Миранчук      24
Обляков       21
Уткин         21
Смолов        20
Глушенков     20
Карраскаль    19
Клаудиньо     17
Name: players, dtype: int64

In [10]:
events_name  = {
    'pass':'пас',
    'goal':'гол',
    'shoot':'удар'
               }

for key in events_name:
#     print(events_name[key])
    df[key] = df['events'].apply(lambda x: 1 if events_name[key] in x else 0)

In [11]:
df.to_csv('players_events_rpl_2022_2023.csv', sep=',', encoding='utf-8')

In [12]:
df.head()

,players,events,pass,goal,shoot
0,Сарвели,Вот это ахнул! Подхватил Сарвели мяч перед чуж...,0,0,0
1,Сарвели,чем надо. Сарвели с правого угла вратарской би...,0,0,0
2,Сарвели,Сарвели отлетает от Зайнутдинова в верховой бо...,0,0,0
3,Сарвели,Снова у нас в центре внимания Сарвели,0,0,0
4,Сарвели,после чего Сарвели справа пытался найти пасом ...,1,0,0
